In [ ]:
import pandas as pd
import numpy as np
import os

In [ ]:
import matplotlib.pyplot as plt
plt.rcParams['font.family'] ='Malgun Gothic'
plt.rcParams['axes.unicode_minus'] =False

In [ ]:
DATASET_DIR = ''
dir_name = './'
file_name = 'BigCitiesHealth.csv'

In [ ]:
file_path = os.path.join(DATASET_DIR,dir_name,file_name)
df = pd.read_csv(file_path)

df.info()

In [ ]:
df["metric_label"] = df["metric_subcat_label"]+' | '+df["metric_item_label"] 

In [ ]:
df["metric_label"].value_counts()

3. value_ci 관련 col 확인

In [ ]:
df['value_ci_flag_yesno'].value_counts()

In [ ]:
np.sum(df['value_ci_flag_yesno'].isna())

In [ ]:
list(filter(lambda x: 'ci' in x,df.columns))

In [ ]:
df[['value_90_ci_high','value_90_ci_low']].info()
# 90% 신뢰구간은 무시해도 상관없음

In [ ]:
df[['value_ci_flag_yesno',
    'value_95_ci_low',
 'value_95_ci_high']]

In [ ]:
#flag = yes 일 경우 low, high 모두 존재
# no 일 경우 최소 1가지 이상 공백
col_list = ['value_95_ci_low',
 'value_95_ci_high']

cond1 = df[col_list].isna().all(axis=1)
cond2 = df['value_ci_flag_yesno'] == 'no'


cond3 = df[col_list].isna().any(axis=1)
cond4 = df['value_ci_flag_yesno'] == 'yes'

# np.sum(cond1 != cond2), np.sum(cond1), np.sum(cond2)

#둘중 하나만 있는 경우, 둘다 있는 경우, flag = yes인 경우
np.sum((~cond3) != cond4), np.sum(~cond3), np.sum(cond4)

In [ ]:
#CI가 없는 컬럼 = 
len(df.loc[cond1,'metric_label'].unique())

In [ ]:
# CI가 존재하는 컬럼 = 53개
len(df.loc[~cond1,'metric_label'].unique())

In [ ]:
df.loc[~cond1,'metric_label'].value_counts()

In [ ]:
metric_w_ci = set(df.loc[~cond1,'metric_label'].unique())
metric_wo_ci = set(df.loc[cond1,'metric_label'].unique())

metric_w_ci.difference(metric_wo_ci)

df[df['metric_label'] == 'Deaths | Premature Death'][col_list]

In [ ]:
len(metric_wo_ci.difference(metric_w_ci)), len(metric_wo_ci)

In [ ]:
metirc_null_ci = sorted(list(metric_wo_ci.difference(metric_w_ci)))

for col in metirc_null_ci[:20]:
    cond = df['metric_label'] == col
    print(col.ljust(75),f"     : {np.sum(cond):>10}")

In [ ]:
for col in metirc_null_ci[20:40]:
    cond = df['metric_label'] == col
    print(col.ljust(75),f"     : {np.sum(cond):>10}")

In [ ]:
for col in metirc_null_ci[40:]:
    cond = df['metric_label'] == col
    print(col.ljust(75),f"     : {np.sum(cond):>10}")

### U.S. Total 분리

In [ ]:
df['geo_label_city'].value_counts()

In [ ]:
cond0 = df['geo_label_city'] == 'U.S. Total'

df_US = df.loc[cond0].copy()
df_city = df.loc[~cond0].copy()

display(df_US.info())
display(df_city.info())

In [ ]:
cond1 = (df_city['strata_race_label'] == 'All') & (df_city['strata_sex_label'] == 'Both')
np.sum(cond1)

In [ ]:
col_list = ['value_95_ci_low',
 'value_95_ci_high',
 'value_90_ci_low',
 'value_90_ci_high']

#전부다 없는 경우
cond1 = df_city[col_list].isna().all(axis=1)

# flag = no 인 경우
cond2 = df_city['value_ci_flag_yesno'] == 'no'

np.sum(cond1 != cond2), np.sum(cond1), np.sum(cond2)

In [ ]:
df_city.loc[cond1,'metric_label'].value_counts()

In [ ]:
#하나라도 있는 경우
df_city.loc[~cond1,'metric_label'].value_counts()

In [ ]:
len(df_city.loc[~cond1,'metric_label'].value_counts())

In [ ]:
metric_w_ci = set(df_city.loc[~cond1,'metric_label'].unique()) # 최소 한개에 대해 CI 있는
metric_wo_ci = set(df_city.loc[cond1,'metric_label'].unique()) # CI 없는

metric_w_ci.difference(metric_wo_ci)

In [ ]:
#모든 레코드에 대해 CI 있는
metirc_full_ci = sorted(list(metric_w_ci.difference(metric_wo_ci)))

for col in metirc_full_ci[:20]:
    cond = df_city['metric_label'] == col
    print(col.ljust(75),f"     : {np.sum(cond):>10}")

In [ ]:
for col in metirc_full_ci[20:]:
    cond = df_city['metric_label'] == col
    print(col.ljust(75),f"     : {np.sum(cond):>10}")

In [ ]:
len(metirc_full_ci)

In [ ]:
# CI 아예 없는 경우
metirc_null_ci = sorted(list(metric_wo_ci.difference(metric_w_ci)))
print(len(metirc_null_ci))

for col in metirc_null_ci:
    cond = df['metric_label'] == col
    print(col.ljust(75),f"     : {np.sum(cond):>10}")

In [ ]:
metirc_null_ci = sorted(list(metric_wo_ci.difference(metric_w_ci)))

for col in metirc_null_ci[20:]:
    cond = df['metric_label'] == col
    print(col.ljust(75),f"     : {np.sum(cond):>10}")

In [ ]:
table = pd.pivot_table(df_city, index=["geo_label_city","date_label"], values='value', columns=['metric_label'])

In [ ]:
table[metirc_null_ci]

In [ ]:
null_count = table.groupby(by=["geo_label_city", "date_label"])[metirc_null_ci].count()

null_count 
# null_count_city = 

In [ ]:
df_city[col_list].isna().astype(int)

In [ ]:
col_list = ["value_95_ci_low", "value_95_ci_high"]

In [ ]:
#CI가 null인것 구하기 위한 함수
def get_nullcount(df,groupby,cols=None):
    grouped = df.groupby(groupby)
    if cols:
        grouped = grouped[cols]
    return grouped.count().rsub(grouped.size(),axis=0)

#CI가 null인것/전체 비율 구하기 위한 함수
def get_nullprop(df,groupby, cols=None):
    grouped = df.groupby(groupby)
    if cols:
        grouped = grouped[cols]
    count = grouped.size().to_frame()
    count = pd.concat([count] * 2, axis=1)
    count.columns = ["value_95_ci_high", "value_95_ci_low"]
    return count.rdiv(get_nullcount(df,groupby,cols=cols))

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
fig, axes = plt.subplots(3,2,figsize=(20,10))

# 연도별 CI가 null 인것 count
null_count_year = get_nullcount(df_city,"date_label",col_list)
null_count_year.plot.bar(ax=axes[0][0])
axes[0][0].set(xlabel="Year", ylabel="Count of null CI")

# 연도 별 CI가 null / 전체 비율
null_prop_year = get_nullprop(df_city,"date_label",col_list) #count_year.rdiv(null_count_year, axis=0)
null_prop_year.plot.bar(ax=axes[1][0])
null_prop_year.plot.box(ax=axes[2][0])
axes[1][0].set(xlabel="Year", ylabel="Proportion of null CI")

# 도시별 CI가 null 인것 count
null_count_city = get_nullcount(df_city,"geo_label_city",col_list) #df_city.groupby("geo_label_city")[col_list].count().rsub(df_city.groupby("geo_label_city")[col_list].size(), axis=0)
null_count_city.plot.bar(ax=axes[0][1])
axes[0][1].set(xlabel="City", ylabel="Count of null CI")

# 도시 별 CI가 null / 전체 비율
null_prop_city = get_nullprop(df_city,"geo_label_city",col_list)
null_prop_city.plot.bar(ax=axes[1][1])
null_prop_city.plot.box(ax=axes[2][1])
axes[1][1].set(xlabel="City", ylabel="Proportion of null CI")

plt.show()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
fig, axes = plt.subplots(3,1,figsize=(6,18))


# Metric_label 별 CI 가 null인것 count
null_count_label = get_nullcount(df_city,"metric_label",cols=col_list)
null_count_label.plot.bar(ax=axes[0])
axes[0].set(xlabel="Label", ylabel="Count of null CI")

# Metric_label 별 CI 가 null인것 비율
null_prop_label = get_nullprop(df_city,"metric_label",cols=col_list) 
null_prop_label.plot.bar(ax=axes[1])
axes[1].set(xlabel="Label", ylabel="Proportion of null CI")

# Metric_label 별 CI 가 null인것 중 0% 또는 100%가 아닌 것의 비율
null_some_label = null_prop_label[(null_prop_label['value_95_ci_high'] != 1.0) & null_prop_label['value_95_ci_high'] != 0.0]
null_some_label.plot.bar(ax=axes[2])
axes[2].set(xlabel="Label", ylabel="Proportion of null CI", title="Proportion of null CI where its not 100% or 0%")

plt.show()

In [ ]:
# ci_low 와 ci_high의 null 값 갯수가 다른것
null_count_label[null_count_label['value_95_ci_high'].notna() & (null_count_label['value_95_ci_high'] != null_count_label['value_95_ci_low'])]

In [ ]:
# 일부 레코드만 CI가 없는 항목들의 갯수 = 19
len(null_some_label.index.to_list())

In [ ]:
null_none_label['value_95_ci_high']

In [ ]:
# CI null 비율이 0/1이 아닌것에 대한 연도/도시 세부 파악

fig, axes = plt.subplots(3,2,figsize=(12,16))
df_somena = df_city[df_city["metric_label"].isin(null_some_label.index.to_list())]
df_all_or_nothing = df_city[~df_city["metric_label"].isin(null_some_label.index.to_list())]


get_nullcount(df_somena, "geo_label_city", cols=col_list).plot.bar(ax=axes[0][0])
get_nullprop(df_somena, "geo_label_city", cols=col_list).plot.bar(ax=axes[0][1])
axes[0][0].set(title="")

get_nullcount(df_all_or_nothing, "geo_label_city", cols=col_list).plot.bar(ax=axes[1][0])
get_nullprop(df_all_or_nothing, "geo_label_city", cols=col_list).plot.bar(ax=axes[1][1])


get_nullcount(df_city, "geo_label_city", cols=col_list).plot.bar(ax=axes[2][0])
get_nullprop(df_city, "geo_label_city", cols=col_list).plot.bar(ax=axes[2][1])


In [ ]:
fig, axes = plt.subplots(2,4,figsize=(24,12),sharey=True)

# CI가 null인 레코드가 하나라도 있으면
null_any_label = null_prop_label[(null_prop_label['value_95_ci_high'] > 0.0)]
null_all_label = null_prop_label[(null_prop_label['value_95_ci_high'] == 1.0)]
null_thresh_label = null_prop_label[(null_prop_label['value_95_ci_high'] > 0.3)]
# CI가 0 이상인 데이터를 다 지웠을떄 분포차이가 생기는지?

df_nona = df_city[~df_city["metric_label"].isin(null_any_label.index.to_list())]
df_nofullna = df_city[~df_city["metric_label"].isin(null_all_label.index.to_list())]

labelwise_counts = df_city.groupby(["metric_label"])["value"].size()
nona_counts = df_nona.groupby(["metric_label"])["value"].size()
# CI가 1인 데이터만 다 지웠을때랑 비교하면?
nofullna_counts = df_nofullna.groupby(["metric_label"])["value"].size()

labelwise_counts.plot.bar(ax=axes[0][0])
axes[0][0].set(title="metric_label별 데이터 분포")
nona_counts.plot.bar(ax=axes[0][1])
axes[0][1].set(title="CI가 하나라도 없는 항목 제거 시 metric_label별 데이터 분포")
nofullna_counts.plot.bar(ax=axes[0][2])
axes[0][2].set(title="전체가 CI 없는 항목 제거 시 metric_label별 데이터 분포")

labelwise_counts.plot.box(ax=axes[1][0])
nona_counts.plot.box(ax=axes[1][1])
nofullna_counts.plot.box(ax=axes[1][2])

df_thresh = df_city[~df_city["metric_label"].isin(null_thresh_label.index.to_list())]
thresh_counts = df_thresh.groupby(["metric_label"])["value"].size()
thresh_counts.plot.bar(ax=axes[0][3])
axes[0][3].set(title="CI 없는 비율이 threshold 이상인 항목 제거 시 metric_label별 데이터 분포")
thresh_counts.plot.box(ax=axes[1][3])


plt.show()

In [ ]:
null_any_label = null_prop_label[(null_prop_label['value_95_ci_high'] > 0.0)]

In [ ]:
null_none_label = null_prop_label[(null_prop_label['value_95_ci_high'] <= 0.0)]
null_none_label['value_95_ci_high']

In [ ]:
pd.set_option('display.max_rows', 65)
null_all_label['value_95_ci_high']

In [ ]:
fig, axes = plt.subplots(3,4,figsize=(24,12))

# CI가 null인 레코드가 하나라도 있으면
null_any_label = null_prop_label[(null_prop_label['value_95_ci_high'] > 0.0)]
# 모든 레코드의 CI가 null이면
null_all_label = null_prop_label[(null_prop_label['value_95_ci_high'] == 1.0)]

# CI null rate가 0 이상인 데이터를 다 지웠을떄 분포차이가 생기는지?
df_nona = df_city[~df_city["metric_label"].isin(null_any_label.index.to_list())]

# CI null rate가 1인 데이터만 다 지웠을때랑 비교하면?
df_nofullna = df_city[~df_city["metric_label"].isin(null_all_label.index.to_list())]

citywise_counts = df_city.groupby(["date_label"])["value"].size()
nona_counts = df_nona.groupby(["date_label"])["value"].size()

nofullna_counts = df_nofullna.groupby(["date_label"])["value"].size()

citywise_counts.plot.bar(ax=axes[0][0])
axes[0][0].set(title="연도별 데이터 분포")
nona_counts.plot.bar(ax=axes[0][1])
axes[0][1].set(title="CI가 하나라도 없는 항목 제거 시 연도별 데이터 분포")
nofullna_counts.plot.bar(ax=axes[0][2])
axes[0][2].set(title="전체가 CI 없는 항목 제거 시 연도별 데이터 분포")

citywise_counts.plot.box(ax=axes[1][0])
nona_counts.plot.box(ax=axes[1][1])
nofullna_counts.plot.box(ax=axes[1][2])

df_thresh = df_city[~df_city["date_label"].isin(null_thresh_label.index.to_list())]
thresh_counts = df_thresh.groupby(["date_label"])["value"].size()
thresh_counts.plot.bar(ax=axes[0][3])
axes[0][3].set(title="CI 없는 비율이 threshold 이상인 항목 제거 시 연도별 데이터 분포")
thresh_counts.plot.box(ax=axes[1][3])

# axes[0][1].sharey(axes[0][0])
# axes[0][2].sharey(axes[0][0])
# axes[0][3].sharey(axes[0][0])

get_nullprop(df_city,"date_label",cols=col_list).plot.bar(ax=axes[2][0])
get_nullprop(df_nona,"date_label",cols=col_list).plot.bar(ax=axes[2][1])
get_nullprop(df_nofullna,"date_label",cols=col_list).plot.bar(ax=axes[2][2])
get_nullprop(df_thresh,"date_label",cols=col_list).plot.bar(ax=axes[2][3])

plt.show()

In [ ]:
get_nullprop(df_city,"date_label",cols=col_list)

In [ ]:
fig, axes = plt.subplots(3,4,figsize=(24,12),sharey=True)

# CI가 null인 레코드가 하나라도 있으면
null_any_label = null_prop_label[(null_prop_label['value_95_ci_high'] > 0.0)]
null_all_label = null_prop_label[(null_prop_label['value_95_ci_high'] == 1.0)]
# CI가 0 이상인 데이터를 다 지웠을떄 분포차이가 생기는지?

df_nona = df_city[~df_city["metric_label"].isin(null_any_label.index.to_list())]
df_nofullna = df_city[~df_city["metric_label"].isin(null_all_label.index.to_list())]

citywise_counts = df_city.groupby(["geo_label_city"])["value"].size()
nona_counts = df_nona.groupby(["geo_label_city"])["value"].size()
# CI가 1인 데이터만 다 지웠을때랑 비교하면?
nofullna_counts = df_nofullna.groupby(["geo_label_city"])["value"].size()

citywise_counts.plot.bar(ax=axes[0][0])

nona_counts.plot.bar(ax=axes[0][1])
axes[0][1].set(title="CI가 하나라도 없는 항목 제거 시 도시별 데이터 분포")
nofullna_counts.plot.bar(ax=axes[0][2])
axes[0][2].set(title="전체가 CI 없는 항목 제거 시 도시별 데이터 분포")

citywise_counts.plot.box(ax=axes[1][0])
nona_counts.plot.box(ax=axes[1][1])
nofullna_counts.plot.box(ax=axes[1][2])

df_thresh = df_city[~df_city["geo_label_city"].isin(null_thresh_label.index.to_list())]
thresh_counts = df_thresh.groupby(["geo_label_city"])["value"].size()
thresh_counts.plot.bar(ax=axes[0][3])
axes[0][3].set(title="CI 없는 비율이 threshold 이상인 항목 제거 시 도시별 데이터 분포")
thresh_counts.plot.box(ax=axes[1][3])

plt.show()

In [ ]:
#값의 분포 확인

from scipy.stats import chi2_contingency

fig, axes = plt.subplots(4,2,figsize=(24,18),sharey=True)

data_somena_city=[]
data_somena_year=[]
data_allna_city=[]
data_allna_year=[]
data_anyna_city=[]
data_anyna_year=[]
data_all_city=[]
data_all_year=[]

for col in null_some_label.index:
    target_df = df_city[df_city["metric_label"] == col]
    chi2,p,dof,ex = chi2_contingency(pd.crosstab(target_df["geo_label_city"],target_df["value"]))
    data_somena_city.append((col,p))

    chi2,p,dof,ex = chi2_contingency(pd.crosstab(target_df["date_label"],target_df["value"]))
    data_somena_year.append((col,p))

for col in null_all_label.index:
    target_df = df_city[df_city["metric_label"] == col]
    chi2,p,dof,ex = chi2_contingency(pd.crosstab(target_df["geo_label_city"],target_df["value"]))
    data_allna_city.append((col,p))

    chi2,p,dof,ex = chi2_contingency(pd.crosstab(target_df["date_label"],target_df["value"]))
    data_allna_year.append((col,p))

for col in null_any_label.index:
    target_df = df_city[df_city["metric_label"] == col]
    chi2,p,dof,ex = chi2_contingency(pd.crosstab(target_df["geo_label_city"],target_df["value"]))
    data_anyna_city.append((col,p))

    chi2,p,dof,ex = chi2_contingency(pd.crosstab(target_df["date_label"],target_df["value"]))
    data_anyna_year.append((col,p))

for col in null_prop_label.index:
    target_df = df_city[df_city["metric_label"] == col]
    chi2,p,dof,ex = chi2_contingency(pd.crosstab(target_df["geo_label_city"],target_df["value"]))
    data_all_city.append((col,p))

    chi2,p,dof,ex = chi2_contingency(pd.crosstab(target_df["date_label"],target_df["value"]))
    data_all_year.append((col,p))

# p < 0.05 -> same distribution

axes[0][0].bar(*zip(*data_somena_city))
axes[0][0].axhline(y=0.05,color='r',linestyle='-')
axes[0][0].set(xlabel="metric_label",ylabel="p-value",title="chi-square homogenieity test across cities for 0<CI null<1")
axes[0][0].tick_params(axis='x', rotation=90)
axes[0][1].bar(*zip(*data_somena_year))
axes[0][1].axhline(y=0.05,color='r',linestyle='-')
axes[0][1].set(xlabel="metric_label",ylabel="p-value",title="chi-square homogenieity test across year for 0<CI null<1")
axes[0][1].tick_params(axis='x', rotation=90)

axes[1][0].bar(*zip(*data_allna_city))
axes[1][0].axhline(y=0.05,color='r',linestyle='-')
axes[1][0].set(xlabel="metric_label",ylabel="p-value",title="chi-square homogenieity test across year for CI null = 1")
axes[1][0].tick_params(axis='x', rotation=90)
axes[1][1].bar(*zip(*data_allna_year))
axes[1][1].axhline(y=0.05,color='r',linestyle='-')
axes[1][1].set(xlabel="metric_label",ylabel="p-value",title="chi-square homogenieity test across year for CI null = 1")
axes[1][1].tick_params(axis='x', rotation=90)

axes[2][0].bar(*zip(*data_anyna_city))
axes[2][0].axhline(y=0.05,color='r',linestyle='-')
axes[2][0].set(xlabel="metric_label",ylabel="p-value",title="chi-square homogenieity test across year for CI null > 0")
axes[2][0].tick_params(axis='x', rotation=90)
axes[2][1].bar(*zip(*data_anyna_year))
axes[2][1].axhline(y=0.05,color='r',linestyle='-')
axes[2][1].set(xlabel="metric_label",ylabel="p-value",title="chi-square homogenieity test across year for CI null > 0")
axes[2][1].tick_params(axis='x', rotation=90)

axes[3][0].bar(*zip(*data_all_city))
axes[3][0].axhline(y=0.05,color='r',linestyle='-')
axes[3][0].set(xlabel="metric_label",ylabel="p-value",title="chi-square homogenieity test across year for CI null > 0")
axes[3][0].tick_params(axis='x', rotation=90)
axes[3][1].bar(*zip(*data_all_year))
axes[3][1].axhline(y=0.05,color='r',linestyle='-')
axes[3][1].set(xlabel="metric_label",ylabel="p-value",title="chi-square homogenieity test across year for CI null > 0")
axes[3][1].tick_params(axis='x', rotation=90)




In [ ]:
# 도시별 분포가 유사한(즉 데이터 분포에 영향을 줄 가능성이 적은) 컬럼들과 연도별 분포가 서로 다른(즉 데이터 분포에 영향 줄 가능성이 큰) 항목들 선별
ambivalence = []
important = []
for col in null_any_label.index:
    target_df = df_city[df_city["metric_label"] == col]
    chi2,p,dof,ex = chi2_contingency(pd.crosstab(target_df["geo_label_city"],target_df["value"]))
    if p > 0.05:
        ambivalence.append(col)

    chi2,p,dof,ex = chi2_contingency(pd.crosstab(target_df["date_label"],target_df["value"]))
    if p < 0.05:
        important.append(col)


# 도시별 분포가 유사한(즉 데이터 분포에 영향을 줄 가능성이 적은) 항목
df_amb = pd.DataFrame(ambivalence)

df_amb

    

In [ ]:
#연도별 분포가 서로 다른(즉 데이터 분포에 영향 줄 가능성이 큰) 항목

df_imp = pd.DataFrame(important)

df_imp

In [ ]:
pd.crosstab(target_df["geo_label_city"],target_df["value"])

In [ ]:
for col in null_any_label.index:
    target_df = df_city[df_city["metric_label"] == col]
    chi2,p,dof,ex = chi2_contingency(pd.crosstab(target_df["geo_label_city"],target_df["value"]))
    print(col, p)

In [ ]:
len(null_some_label)

In [ ]:
citywise_counts

In [ ]:
# 집계 년도가 격년인지 확인하는 함수 정의
def is_biennial(df, year_column='date_label'):
    # 연도 리스트 추출 및 정렬
    years = sorted(df[year_column].unique())
    
    # 연도 간 차이 계산
    differences = [years[i+1] - years[i] for i in range(len(years) - 1)]
    
    # 모든 차이가 2년인지 확인
    return all(diff == 2 for diff in differences)

# 함수 사용
if is_biennial(df_city[df_city["metric_label"] == "Mental Health | Teen Mental Distress"]):
    print("집계 년도가 격년입니다.")
else:
    print("집계 년도가 격년이 아닙니다.") 

In [ ]:
null_some_label.index

In [ ]:
fig, ax = plt.subplots(figsize=(6,8))
ax.hist(x = df_city[df_city["metric_label"].isin(null_some_label.index)]["date_label"])

In [ ]:
df_city[df_city["metric_label"].isin(null_some_label.index)]["date_label"].value_counts()

In [ ]:
display(df_city[df_city["metric_label"].isin(null_some_label.index)]["strata_race_label"].value_counts())
display(df_city[df_city["metric_label"].isin(null_some_label.index)]["strata_sex_label"].value_counts())

In [ ]:
df_city[(df_city["metric_label"].isin(null_some_label.index)) & (df_city["strata_race_label"] == "All") & (df_city["strata_sex_label"] == "Both")]["date_label"].plot.hist()

In [ ]:
df_city[(df_city["metric_label"].isin(null_some_label.index)) & (df_city["strata_race_label"] == "All") & (df_city["strata_sex_label"] == "Both")]["date_label"].value_counts()

In [ ]:
df_city[(df_city["metric_label"].isin(null_some_label.index)) & (df_city["strata_race_label"] == "All") & (df_city["strata_sex_label"] == "Both")].groupby("metric_label")["geo_label_city"].value_counts()
# 19 * 11 -> 19 column, 11 years (2021/2022 missing)
# only in 19 cities, and LV has half data
# LV has Diabetes and Obesity | Teen Obesity, 

In [ ]:
pd.set_option('display.max_rows', None)
df_city[(df_city["metric_label"].isin(null_some_label.index)) & (df_city["strata_race_label"] == "All") & (df_city["strata_sex_label"] == "Both")].groupby(["metric_label","date_label","geo_label_city"])["value"].count()

In [ ]:
teen_index = ["Mental Health | Teen Suicidal Ideation",

"Physical Activity | Teen Computer Time",

"Physical Activity | Teen Physical Activity",

"Physical Activity | Teen TV Time",

"Respiratory Disease | Teen Asthma",

"Substance Use | Teen Alcohol",

"Substance Use | Teen Marijuana",

"Substance Use | Teen Smoking",

"Diabetes and Obesity | Teen Obesity"]

In [ ]:
df_city[(df_city["metric_label"].isin(teen_index)) & (df_city["strata_race_label"] == "All") & (df_city["strata_sex_label"] == "Both")].groupby("geo_label_city")["metric_label"].value_counts()

In [ ]:
teen_index = ["Physical Activity | Teen Computer Time",

"Physical Activity | Teen Physical Activity",

"Physical Activity | Teen TV Time",

"Respiratory Disease | Teen Asthma",

"Substance Use | Teen Alcohol",

"Substance Use | Teen Marijuana",

"Substance Use | Teen Smoking",

"Diabetes and Obesity | Teen Obesity"]

In [ ]:
df_teen = df_city[(df_city["metric_label"].isin(teen_index)) & (df_city["strata_race_label"] == "All") & (df_city["strata_sex_label"] == "Both")]

table_teen = pd.pivot_table(df_teen, values="value", index=["date_label","geo_label_city"], columns="metric_label")

fig, ax = plt.subplots(figsize=(20,20))
table_teen.plot.bar(ax=ax)

In [ ]:
df_teen

In [145]:
pd.set_option('display.max_rows', 65)
display(df_city[df_city["metric_label"].isin(teen_index)]["strata_race_label"].value_counts())
display(df_city[df_city["metric_label"].isin(teen_index)]["strata_sex_label"].value_counts())

All         2673
Hispanic     498
Black        447
White        410
Asian/PI     320
Name: strata_race_label, dtype: int64

Both      3336
Male       506
Female     506
Name: strata_sex_label, dtype: int64

In [156]:
display(df_city[df_city["metric_label"].isin(ambivalence)]["strata_race_label"].value_counts())
display(df_city[df_city["metric_label"].isin(ambivalence)]["strata_sex_label"].value_counts())

All         2622
White       1260
Black       1179
Hispanic     808
Asian/PI     485
Name: strata_race_label, dtype: int64

Both      3122
Male      1646
Female    1586
Name: strata_sex_label, dtype: int64

In [161]:
display(df_city[df_city["metric_label"].isin(important)]["strata_race_label"].value_counts())
display(df_city[df_city["metric_label"].isin(important)]["strata_sex_label"].value_counts())

All                              2603
White                             795
Hispanic                          775
Black                             775
Asian/PI                          385
Asian                             373
American Indian/Alaska Native      94
Name: strata_race_label, dtype: int64

Both      4980
Male       410
Female     410
Name: strata_sex_label, dtype: int64

In [160]:
df_city[df_city["metric_label"].isin(ambivalence) & (df_city["strata_race_label"] == "All") & (df_city["strata_sex_label"] == "Both")].groupby("date_label")["metric_label"].count()

date_label
2010     35
2011     70
2012     70
2013     70
2014     70
2015    168
2016    168
2017    166
2018    166
2019    166
2020    164
2021    234
2022    235
Name: metric_label, dtype: int64